### Assignment 1: Creating kNN Algorithm
Student Name: 류정민 #2023029516

In [1]:
import numpy as np
import torch
from torchvision import datasets
trainset = datasets.MNIST(root='./data', train=True, download=True)
testset = datasets.MNIST(root='./data', train=False, download=True)

In [2]:
# Indices for train/val splits: train_idx, valid_idx
np.random.seed(0)
val_ratio = 0.1
train_size = len(trainset)
indices = list(range(train_size))
split_idx = int(np.floor(val_ratio * train_size)) 
np.random.shuffle(indices)
train_idx , val_idx = indices[split_idx:], indices[:split_idx]
train_data = trainset.data[train_idx].float()/255. 
train_labels = trainset.targets[train_idx] 
val_data = trainset.data[val_idx].float()/255.
val_labels = trainset.targets[val_idx]
test_data = testset.data.float()/255. 
test_labels = testset.targets

### Problem a

In [3]:
# Problem a

# function for calculating Euclidean distance
def cal_dst_L2(a,b):
  return (a-b)**2

def knn(train_data, target, k):
  # list to contain information about distance
  li = []

  # KNN in for loop
  for i in range(len(train_data)):
    sample = train_data[i]
    # distance for each instance
    dist = 0
    for j in range(28):
      for l in range(28):
        dist += cal_dst_L2(float(target[j][l]),float(sample[j][l]))
    # append information of distance to the list
    li.append(dist)

  # counts each labels
  cnt = {}

  for i in range(k):
    # extract minimum value
    ele = min(li)
    # extract minimum value's index
    idx = li.index(ele)
    # remove minimum value from the list
    li.remove(ele)

    # count the value
    temp = int(train_labels[idx])
    if temp in cnt:
      cnt[temp] += 1
    else:
      cnt[temp] = 1

  # extract the maximum counted value from the dictionary
  knn = max(cnt, key = lambda x: cnt[x])

  # return the knn result
  return knn


In [4]:
# Testing if a works

# first, choose a sample target from the test_data (unseen)
target = test_data[5]

# perform knn with k = 5
res = knn(train_data, target, 5)

print(res)
print(test_labels[5])

1
tensor(1)


### Problem b

In [5]:
# Problem b

def knn_broadcast(train_data, target, k):

  # Use broadcasting method to calculate distance between each element in train data and the target
  knn_data = (train_data - target)**2

  # list to contain information about distance
  li = []

  # Distance Calculation
  for i in range(len(knn_data)):
    # calculate the distance of the instance
    dist = torch.sum(knn_data[i])
    # add the distance to the list
    li.append((dist, i))

  # counts each labels
  cnt = {}

  for i in range(k):
    # extract minimum value
    ele = min(li)
    # extract minimum value's index
    idx = li.index(ele)
    # remove minimum value from the list
    li.remove(ele)

    # count the value
    temp = int(train_labels[idx])
    if temp in cnt:
      cnt[temp] += 1
    else:
      cnt[temp] = 1

  # sort the dictionary with the value
  knn = max(cnt, key = lambda x: cnt[x])
  
  # return the last element after sorting incrementally -> returns greatest element
  return knn

In [6]:
# Testing if it works

# first, choose a sample target from the test_data (unseen)
target = test_data[5]

# perform knn with k = 5
res = knn_broadcast(train_data, target, 5)

print(res)
print(test_labels[5])

1
tensor(1)


### Problem c

In [7]:
# Question c

# Function to calculate the accuracy of the model
def acc(res, test_labels):
  cnt = 0
  length = len(test_labels)
  for i in range(length):
    if test_labels[i] == res[i]:
      cnt += 1
  return "Test Accuracy: {}".format(cnt/length*100)


def knn_broadcast_all(train_data, target, test_labels, k):

  # A dictionary that contains target's label and its classified result
  res = {}

  for i in range(len(target)):

    # Using broadcasting method to calculate the Euclidean Distance
    knn_data = (train_data - target[i])**2
    
    # sum the distance using torch.sum with column direction twice to get 1D tensor
    dist1 = torch.sum(knn_data, dim = 1)
    dist2 = torch.sum(dist1, dim = 1)

    # counts each labels
    cnt = {}

    # changed iterable variable to avoid crashing with the outer for loop
    for l in range(k):
      
      # change python list operation to pytorch operation to employ vector operation to reduce time
      ele, idx = torch.min(dist2, dim=0)
      # Change the value to a sentinel
      dist2[idx] = float('inf')

      # Count the labels
      temp = int(train_labels[idx])
      if temp in cnt:
        cnt[temp] += 1
      else:
        cnt[temp] = 1

    # Extract the key where its value has the greatest counts
    knn = max(cnt, key = lambda x: cnt[x])
    
    # Add the index and its classified result into the dictionary
    res[i] = knn
  
  # return the accuracy of the model
  return acc(res, test_labels)

In [14]:
# Testing if it works

# This time, the entire test_data set is our target data
target = test_data

# perform knn with k = 5
res = knn_broadcast_all(train_data, target, test_labels, 5)

print(res)

Test Accuracy: 96.74000000000001


### Problem d

In [8]:
# Question d

def acc(res, test_labels):
  cnt = 0
  length = len(test_labels)
  for i in range(length):
    if test_labels[i] == res[i]:
      cnt += 1
  return "Test Accuracy: {}".format(cnt/length*100)

# Here, the target is the entire test_data
def knn_broadcast_all_d_ma(train_data, target, test_labels, k):

  num = len(target)

  train_data = torch.flatten(train_data, start_dim=1).unsqueeze(0)
  target = torch.flatten(target, start_dim=1).unsqueeze(0)

  dist = torch.cdist(train_data, target, p=2, compute_mode="use_mm_for_euclid_dist_if_necessary")

  # A dictionary that contains target's label and its classified result
  res = {}

  for i in range(num):
    # counts each labels
    cnt = {}

    # changed iterable variable to avoid crashing with the outer for loop
    for l in range(k):
      
      # change python list operation to pytorch operation to employ vector operation to reduce time
      ele, idx = torch.min(dist[0,:,i], dim=0)
      dist[0,idx,i] = float('inf')

      temp = int(train_labels[idx])
      if temp in cnt:
        cnt[temp] += 1
      else:
        cnt[temp] = 1

    # sort the dictionary with the value
    knn = max(cnt, key = lambda x: cnt[x])
    
    # Add the index and its classified result into the dictionary
    res[i] = knn
  
  return acc(res, test_labels)

In [9]:
# Testing if it works

# This time, the entire test_data set is our target data
target = test_data

# perform knn with k = 5
res = knn_broadcast_all_d_ma(train_data, target, test_labels, 5)

print(res)

Test Accuracy: 96.74000000000001


### Problem e

The hyperparameters I can tune are distance and the k value

### Problem f

In [10]:
# Question d

def acc(res, test_labels):
  cnt = 0
  length = len(test_labels)
  for i in range(length):
    if test_labels[i] == res[i]:
      cnt += 1
  return "Test Accuracy: {}".format(cnt/length*100)

# Here, the target is the entire test_data
def knn_broadcast_all_d_ma_p(train_data, target, test_labels, k, p_value):

  num = len(target)

  train_data = torch.flatten(train_data, start_dim=1).unsqueeze(0)
  target = torch.flatten(target, start_dim=1).unsqueeze(0)

  dist = torch.cdist(train_data, target, p=p_value, compute_mode="use_mm_for_euclid_dist_if_necessary")

  # A dictionary that contains target's label and its classified result
  res = {}

  for i in range(num):
    # counts each labels
    cnt = {}

    # changed iterable variable to avoid crashing with the outer for loop
    for l in range(k):
      
      # change python list operation to pytorch operation to employ vector operation to reduce time
      ele, idx = torch.min(dist[0,:,i], dim=0)
      dist[0,idx,i] = float('inf')

      temp = int(train_labels[idx])
      if temp in cnt:
        cnt[temp] += 1
      else:
        cnt[temp] = 1

    # sort the dictionary with the value
    knn = max(cnt, key = lambda x: cnt[x])
    
    # Add the index and its classified result into the dictionary
    res[i] = knn
  
  return acc(res, test_labels)

In [11]:
# Testing if it works

# This time, the entire test_data set is our target data
target = test_data

# perform knn with k = 5
res1 = knn_broadcast_all_d_ma_p(train_data, target, test_labels, 8, 2)
#res2 = knn_broadcast_all_d_ma_p(train_data, target, test_labels, 1, 2)
#res2 = knn_broadcast_all_d_ma_p(train_data, target, test_labels, 5, 3)
#res2 = knn_broadcast_all_d_ma_p(train_data, target, test_labels, 5, 1)

print(res)

Test Accuracy: 96.74000000000001


In [12]:
# Testing if it works

# This time, the entire test_data set is our target data
target = test_data

# perform knn with k = 5
#res1 = knn_broadcast_all_d_ma_p(train_data, target, test_labels, 8, 2)
res2 = knn_broadcast_all_d_ma_p(train_data, target, test_labels, 1, 2)
#res2 = knn_broadcast_all_d_ma_p(train_data, target, test_labels, 5, 3)
#res2 = knn_broadcast_all_d_ma_p(train_data, target, test_labels, 5, 1)

print(res)

Test Accuracy: 96.74000000000001


In [13]:
# Testing if it works

# This time, the entire test_data set is our target data
target = test_data

# perform knn with k = 5
#res1 = knn_broadcast_all_d_ma_p(train_data, target, test_labels, 8, 2)
#res2 = knn_broadcast_all_d_ma_p(train_data, target, test_labels, 1, 2)
res2 = knn_broadcast_all_d_ma_p(train_data, target, test_labels, 5, 3)
#res2 = knn_broadcast_all_d_ma_p(train_data, target, test_labels, 5, 1)

print(res)